In [1]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 


from sklearn.preprocessing import scale



# Pre-processing

Some steps may be needed before applying custering algorithms

## Load Data

In [3]:
dat = pd.read_csv("/Users/jesus/Dropbox/Planes/proyectos/academia/IE/2021-2022/ML/dat/i2.csv", sep = ";")
dat.head()

,user,booking_date,origin_airport,price,sales channel,ant
0,user5,2018-11-01,MAD,58.200001,online,7
1,user7,2018-11-01,DUB,147.500000,online,38
2,user4,2018-11-02,TFS,24.049999,online,19
3,user8,2018-10-29,MAD,59.709999,online,8
4,user7,2018-11-01,LPA,37.299999,call center,4


## One-hot Encoding

<img src="figures/oh.png" width="50%">

Optional. Only if we want to apply K-means directly to categorical variables without using k-prototypes. Less optimal than using K-prototypes without one-hot encoding.

In [9]:
from sklearn.preprocessing import OneHotEncoder
?OneHotEncoder

In [ ]:
dat.dtypes

In [8]:
categorical_vars = set(dat.columns[dat.dtypes == object])
numerical_vars = set(dat.columns) - categorical_vars
categorical_vars = list(categorical_vars)
numerical_vars = list(numerical_vars)
print(categorical_vars)
print(numerical_vars)

['sales channel', 'origin_airport', 'booking_date', 'user']
['price', 'ant']


In [12]:
ohe = OneHotEncoder(sparse = False)
ohe_fit = ohe.fit(dat[categorical_vars])
dat_ohe = pd.DataFrame(ohe.fit_transform(dat[categorical_vars]))
dat_ohe.columns = pd.DataFrame(ohe_fit.get_feature_names())

In [13]:
dat[categorical_vars].head()

,sales channel,origin_airport,booking_date,user
0,online,MAD,2018-11-01,user5
1,online,DUB,2018-11-01,user7
2,online,TFS,2018-11-02,user4
3,online,MAD,2018-10-29,user8
4,call center,LPA,2018-11-01,user7


In [14]:
dat_ohe.head()

,"(x0_call center,)","(x0_online,)","(x0_travel agency,)","(x1_ACE,)","(x1_AGP,)","(x1_ALC,)","(x1_AMS,)","(x1_BHX,)","(x1_BOD,)","(x1_BPS,)",...,"(x3_user1,)","(x3_user10,)","(x3_user2,)","(x3_user3,)","(x3_user4,)","(x3_user5,)","(x3_user6,)","(x3_user7,)","(x3_user8,)","(x3_user9,)"
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
dat = pd.concat((dat_ohe, dat[numerical_vars].reset_index()), axis=1)

In [18]:
dat

,"(x0_call center,)","(x0_online,)","(x0_travel agency,)","(x1_ACE,)","(x1_AGP,)","(x1_ALC,)","(x1_AMS,)","(x1_BHX,)","(x1_BOD,)","(x1_BPS,)",...,"(x3_user3,)","(x3_user4,)","(x3_user5,)","(x3_user6,)","(x3_user7,)","(x3_user8,)","(x3_user9,)",index,price,ant
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,58.200001,7
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,147.500000,38
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,24.049999,19
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,59.709999,8
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,37.299999,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,995,99.849998,29
996,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,996,34.610001,39
997,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,997,49.880001,5
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,998,152.960007,4


In [17]:
dat.dtypes

(x0_call center,)      float64
(x0_online,)           float64
(x0_travel agency,)    float64
(x1_ACE,)              float64
(x1_AGP,)              float64
                        ...   
(x3_user8,)            float64
(x3_user9,)            float64
index                    int64
price                  float64
ant                      int64
Length: 74, dtype: object

## Scaling (Tipifying)

Mandatory and critical. Really important to achieve a good fit of your clustering algorithm due to use of distances.

Let's scale to zero mean and standard deviation 1.

<img src="figures/tipify.png" width="50%">

In [19]:
dat_scale = pd.DataFrame(scale(dat))
dat_scale.columns = dat.columns

In [20]:
print(dat.head())

   (x0_call center,)  (x0_online,)  (x0_travel agency,)  (x1_ACE,)  (x1_AGP,)  \
0                0.0           1.0                  0.0        0.0        0.0   
1                0.0           1.0                  0.0        0.0        0.0   
2                0.0           1.0                  0.0        0.0        0.0   
3                0.0           1.0                  0.0        0.0        0.0   
4                1.0           0.0                  0.0        0.0        0.0   

   (x1_ALC,)  (x1_AMS,)  (x1_BHX,)  (x1_BOD,)  (x1_BPS,)  ...  (x3_user3,)  \
0        0.0        0.0        0.0        0.0        0.0  ...          0.0   
1        0.0        0.0        0.0        0.0        0.0  ...          0.0   
2        0.0        0.0        0.0        0.0        0.0  ...          0.0   
3        0.0        0.0        0.0        0.0        0.0  ...          0.0   
4        0.0        0.0        0.0        0.0        0.0  ...          0.0   

   (x3_user4,)  (x3_user5,)  (x3_user6,)  (x

In [21]:
print(dat_scale.head())

   (x0_call center,)  (x0_online,)  (x0_travel agency,)  (x1_ACE,)  (x1_AGP,)  \
0          -0.193247      0.772945            -0.714545  -0.100504  -0.131507   
1          -0.193247      0.772945            -0.714545  -0.100504  -0.131507   
2          -0.193247      0.772945            -0.714545  -0.100504  -0.131507   
3          -0.193247      0.772945            -0.714545  -0.100504  -0.131507   
4           5.174725     -1.293753            -0.714545  -0.100504  -0.131507   

   (x1_ALC,)  (x1_AMS,)  (x1_BHX,)  (x1_BOD,)  (x1_BPS,)  ...  (x3_user3,)  \
0  -0.119159   -0.14646  -0.031639  -0.031639  -0.031639  ...    -0.349763   
1  -0.119159   -0.14646  -0.031639  -0.031639  -0.031639  ...    -0.349763   
2  -0.119159   -0.14646  -0.031639  -0.031639  -0.031639  ...    -0.349763   
3  -0.119159   -0.14646  -0.031639  -0.031639  -0.031639  ...    -0.349763   
4  -0.119159   -0.14646  -0.031639  -0.031639  -0.031639  ...    -0.349763   

   (x3_user4,)  (x3_user5,)  (x3_user6,)  (x